In [13]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pyautogui
import re
from bs4 import BeautifulSoup
import sys
sys.path.append('..')
import constants.constants as const
import constants.file_handler_constants as fh
from constants.restaurant_constants import *

from packages.restaurant.Restaurant import *
from packages.file_handler_package.file_handler import *

import os
import glob
import time
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver import ActionChains
# from selenium import webdriver
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.remote.webelement import WebElement


In [14]:
def scrape_img():
    pass

In [15]:
def scrape_location():
    pass

In [16]:
def scrape_restaurant(link_to_restaurant: str, restaurant: Restaurant) -> None:
    # navigate to the current restaurant page and continue scraping more data
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")

    restaurant_page_driver = webdriver.Chrome(options=options) 
    restaurant_page_driver.get(link_to_restaurant)

    # find websites
    all_website_dict = {}
    try:
        container_website_elements = restaurant_page_driver.find_element(By.CLASS_NAME, 'kKDiaN')
        all_website_elements = container_website_elements.find_elements(By.CLASS_NAME, 'cXFOMU')
        for cur_website in all_website_elements:
            cur_website_name = cur_website.text
            cur_website_link = cur_website.get_attribute('href')
            all_website_dict[cur_website_name] = cur_website_link

    except Exception as e:
        print("no website ...")

    print(all_website_dict)

    # find price range
    priceRange = ""
    try:
        possible_priceRange_elements = restaurant_page_driver.find_elements(By.CLASS_NAME, 'hpJBMe')
        for cur_element in possible_priceRange_elements:
            cur_text = cur_element.text
            if("บาท" in cur_text):
                priceRange = cur_text.replace('(', '').replace(')', '')
                break
    except Exception as e:
        print("no price range ...")        
    
    print(priceRange)

    # find facilities
    facilities = []
    try:
        container_list_facilities = restaurant_page_driver.find_element(By.XPATH, '//*[@id="contentRow"]/div[2]/div/div[1]/div[2]/div/ul')
        all_list_facilities = container_list_facilities.find_elements(By.TAG_NAME, 'li')

        for cur_list_element in all_list_facilities:
            all_span_elements = cur_list_element.find_elements(By.TAG_NAME, 'span')
            allowed_className = "buIyWl"
            # not_allowed_className = "McJoy"
            for cur_span in all_span_elements:
                cur_class =  cur_span.get_attribute('class')
                is_allowed_facility_span = (allowed_className in cur_class)

                # check if is facility with correct mark symbol -> if it is then continue scrape for facility
                if(is_allowed_facility_span):
                    cur_text = cur_list_element.find_element(By.CLASS_NAME, 'fFYUJu').text
                    try:
                        cur_sub_text = cur_list_element.find_element(By.CLASS_NAME, 'gFBGSr').text
                        cur_text = ('%s %s') % (cur_text, cur_sub_text)
                    except Exception as e:
                        pass
                    
                    facilities.append(cur_text)
                    break

        print("cur facilities -> ", facilities)

    except Exception as e:
        print("no facilities ...")

    
    # find openingHours
    openingHours = {}
    try:
        all_openingHours_element = restaurant_page_driver.find_elements(By.CLASS_NAME, 'gdNTro')
        for cur_openingHours_element in all_openingHours_element:
            cur_all_td_elements = cur_openingHours_element.find_elements(By.TAG_NAME, 'td')
            cur_day = cur_all_td_elements[0].text
            cur_time = cur_all_td_elements[1].text
            openingHours[cur_day] = cur_time

    except Exception as e:
        print("no opening hours ...")

    print("cur opening hours: ")
    print(openingHours)
 
    # scrape location

    # scrape image path

    restaurant_page_driver.close()

In [17]:
def scrape_restaurants_by_province(province: str, wongnai_regionId: str) -> pd.DataFrame:
    # prepare option to create new driver for some scraping task
    # for example: get to google map or next page without click on the same browser(wongnai might not allowed) 
    # options = webdriver.ChromeOptions()
    # options.add_argument("start-maximized")
    
    res_restaurant_df = pd.DataFrame()
    is_last_page = False
    cur_query_url = "https://www.wongnai.com/restaurants?categoryGroupId=9&regions=%s" % (wongnai_regionId)
    
    # create webdriver instance using option to mazimize current window
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")

    cnt_for_debug = 0
    while(not is_last_page):
        if(cnt_for_debug == 2):
            break
        cnt_for_debug += 1

        driver = webdriver.Chrome(options=options)
        driver.get(cur_query_url)
        
        # find first group of restaurant on the first page
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'dibyTT')))
        all_restaurants_card = driver.find_elements(By.CLASS_NAME, 'dibyTT')
        all_clickable_elements = driver.find_elements(By.CLASS_NAME, 'fsElrZ')

        # iterate to scrape each restaurant and set propeties of 'Restaurant' object
        for cur_restaurant_card, cur_clickable_element in zip(all_restaurants_card, all_clickable_elements):
            cur_restaurant = Restaurant()

            # find restaurant name
            cur_name = cur_restaurant_card.find_element(By.CLASS_NAME, 'Dtkmv').text
            cur_sub_name = ''
            # cut substring if there is sub name
            try:
                cur_sub_name = cur_restaurant_card.find_element(By.CLASS_NAME, 'dqdias').text
                print("sub name -> ", cur_sub_name)
                cur_Idx_sub_name = cur_name.rfind(cur_sub_name)
                cur_name = cur_name[:cur_Idx_sub_name]
                
            except Exception as e:
                pass

            print('name -> ', cur_name)
            cur_restaurant.set_name(cur_name)
            cur_restaurant.set_sub_name(cur_sub_name)

            # find restaurant tags
            cur_restaurant_tags = []
            cur_restaurant_tags_elements = cur_restaurant_card.find_elements(By.CLASS_NAME, 'okmRN')
            for cur_element in cur_restaurant_tags_elements:
                cur_restaurant_tags.append(cur_element.text)
            print("cur tags")
            print(cur_restaurant_tags)

            # set some properties of 'Restaurant' object
            cur_restaurant.set_name(cur_name)
            cur_restaurant.set_sub_name(cur_sub_name)
            cur_restaurant.set_restaurantTag(cur_restaurant_tags)

            time.sleep(2)
            # navigate to the current restaurant page and continue scraping more data
            # get url via "href" attribute of tag 'a' -> navigate restaurant page
            link_to_restaurant = cur_clickable_element.get_attribute('href')
            scrape_restaurant(
                link_to_restaurant = link_to_restaurant,
                restaurant = cur_restaurant,
            )
            
        # find button to click next -> get url via "href" attribute of tag 'a' -> navigate to the next page in a new browser window (aviod wongnai might not allowed web scraping).
        time.sleep(2)
        next_page_element = driver.find_element(By.XPATH, '//*[@id="contentContainer"]/div[1]/div[2]/div[2]/div[1]/div[2]/a[2]')
        link_to_nextPage = next_page_element.get_attribute('href')
        # after scrap everything in current page -> check if it is the lastpage
        if(not link_to_nextPage):
            break

        # if not last page -> set new query to navigate to the next page in a new browser window(next iteration)
        cur_query_url = link_to_nextPage
        # close current browser tab    
        driver.close() 

    return res_restaurant_df.copy()


In [18]:
cnt_for_debug = 0
for cur_region_data in ALL_REGION_ID_WONGNAI:
    if(cnt_for_debug == 1):
        break
    
    cur_province = cur_region_data[1]
    cur_regionId = cur_region_data[2]

    # get dataframe result of all restaurants in current province
    cur_res_allRestaurants_df = scrape_restaurants_by_province(
        wongnai_regionId = cur_regionId,
        province = cur_province
    )
    cnt_for_debug += 1
    # save result dataframe to .csv
    

name ->  นามิ ซูชิ - Nami Sushi Phuket
cur tags
['อาหารญี่ปุ่น']
no website ...
{}
101 - 250 บาท
cur facilities ->  ['ที่จอดรถ (มีที่จอดรถ)', 'รับบัตรเครดิต', 'เดลิเวอรี']
cur opening hours: 
{'ทุกวัน': '10:30 - 21:00'}
sub name ->  Phuket
name ->  Wood x Good Cafe คาเฟ่ภูเก็ต
cur tags
['เบเกอรี/เค้ก', 'คาเฟ่']
{'Facebook': 'https://www.facebook.com/woodxgoodcafe', 'Instagram': 'https://instagram.com/https://www.instagram.com/woodxgood_cafe/', 'LINE@': 'http://line.naver.jp/ti/p/@woodxgoodcafe'}
ต่ำกว่า 100 บาท
cur facilities ->  ['ที่จอดรถ (มีที่จอดรถ)', 'เดลิเวอรี']
cur opening hours: 
{'ทุกวัน': '08:00 - 20:00'}
sub name ->  Patong
name ->  Napa Massage
cur tags
['คาเฟ่']
no website ...
{}
101 - 250 บาท
no facilities ...
cur opening hours: 
{}
sub name ->  สามกอง ภูเก็ต
name ->  Pizza Hut
cur tags
['พิซซ่า', 'ฟาสต์ฟู้ด/จานด่วน']


KeyboardInterrupt: 